# Tarefa 1

Aluno: Artur Ricardo Bizon

## Passo 1 
Inicialmente é realizado o download do arquivo pcap presente no repositório do github, então é instalada a biblioteca scapy

In [3]:
!wget https://raw.github.com/ArturBizon/data-science-seguranca/main/tarefas/Tarefa1/Dados/pacotes.pcap
!pip install scapy

--2022-02-16 21:54:04--  https://raw.github.com/ArturBizon/data-science-seguranca/main/tarefas/Tarefa1/Dados/pacotes.pcap
Resolving raw.github.com (raw.github.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.github.com (raw.github.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/ArturBizon/data-science-seguranca/main/tarefas/Tarefa1/Dados/pacotes.pcap [following]
--2022-02-16 21:54:04--  https://raw.githubusercontent.com/ArturBizon/data-science-seguranca/main/tarefas/Tarefa1/Dados/pacotes.pcap
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1817058 (1.7M) [application/octet-stream]
Saving to: ‘pacotes.pcap’

pacotes.pcap        100

## Passo 2
Após a instalação, é feito a importação da biblioteca e a definição das funções que serão utilizadas

In [5]:
from scapy.all import *

def get_number_of_packets_for_each_ip_protocol(sessionsDict):
  protocolsDict = {}
  for key in sessionsDict:
    protocol = key.split(" ")[0]
    nPacks = len(sessionsDict[key])
    if not protocol in protocolsDict:
      protocolsDict[protocol] = nPacks
    else:
      protocolsDict[protocol] += nPacks
  
  return protocolsDict

def create_readable_session_dict(sessions):
  sessionsDict = {}
  for session in sessions:
    sessionsDict[session] = []
    for packet in sessions[session]:
      sessionsDict[session].append(bytes(packet.payload).decode("ascii",
                                                                "backslashreplace"))
  return sessionsDict


def get_ip_packets(packets):
  return list(filter(lambda x: IP in x, packets))

def get_no_ip_packets(packets):
  return list(filter(lambda x: not IP in x, packets))


## Passo 3
Neste momento os dados são carregados e então são exibidas algumas informações básicas sobre os dados, como o número de pacotes e de sessões

In [6]:
pcapFilePath = "pacotes.pcap"
packetsData = rdpcap(pcapFilePath)

ipPackets = get_ip_packets(packetsData)
noIpPackets = get_no_ip_packets(packetsData)

sessions = packetsData.sessions()
sumDict = get_number_of_packets_for_each_ip_protocol(sessions)

mappedProtocols = list(sumDict)
nSessionDict = {"UDP":0, "TCP":0}
for session in sessions:
  if session.startswith("UDP"):
    nSessionDict["UDP"] += 1
  
  if session.startswith("TCP"):
    nSessionDict["TCP"] += 1

print("O arquivo possui {} pacotes no total \n".format(len(packetsData)))
print("O arquivo possui {} pacotes IP \n".format(len(ipPackets)))
print("O arquivo possui {} pacotes não IP \n".format(len(noIpPackets)))

for protocol in sumDict:
  print("O arquivo possui {} pacotes {} \n".format(sumDict[protocol], protocol))  

for key in nSessionDict:
  print("O arquivo possui {} sessoes {} \n".format(nSessionDict[key], key))  

O arquivo possui 3090 pacotes no total 

O arquivo possui 3086 pacotes IP 

O arquivo possui 4 pacotes não IP 

O arquivo possui 1852 pacotes UDP 

O arquivo possui 1236 pacotes TCP 

O arquivo possui 2 pacotes ARP 

O arquivo possui 115 sessoes UDP 

O arquivo possui 48 sessoes TCP 



## Passo 4
No código abaixo, é criado um dicionário que contém os _payloads_ dos pacotes enviados em cada sessão

In [7]:
sessionsData = create_readable_session_dict(sessions)
for session in sessionsData:
  print("============== Sessao {} ==================".format(session))
  for packet in sessionsData[session]:
    print(packet)
    print()
  

============== Sessao UDP 10.0.2.15:42655 > 192.168.0.1:53 ==================
E  V\xcaQ@ @\xa3\x8d
 \xc0\xa8 \xa6\x9f 5 B\xcd\x97\x85       clientservices
googleapiscom     )       

============== Sessao UDP 192.168.0.1:53 > 10.0.2.15:42655 ==================
E  f 6  @\xad\x99\xc0\xa8 
  5\xa6\x9f R\x84\xd0\x97\x85\x81\x80     clientservices
googleapiscom   \xc0     \xc2 \x8e\xfb\x80\x83  )       

============== Sessao TCP 10.0.2.15:57028 > 142.251.128.131:443 ==================
E  <3@ @\xec
 \x8e\xfb\x80\x83\xde\xc4\xbb\x91`"\x94    \xa0\xfa\xf0\xbc  \xb4
\xbc\xc8\x9e    

E  (3@ @\xec(
 \x8e\xfb\x80\x83\xde\xc4\xbb\x91`"\x95 ".P\xfa\xf0\xa8  

E -3@ @\xea"
 \x8e\xfb\x80\x83\xde\xc4\xbb\x91`"\x95 ".P\xfa\xf0\xad    \xfc\x96%\xc7x\x94\\xa974\xe0CX\xbc\xa1x:\xd5\xc4d\xf0\xd6\x8c	\xf2\xb1H\xb7Y \xabIqAO!`\xb8\x9b\x94\xe7"\xe7\xf2^\xec\xc6z\xc3\x89iL\xd3\xd3#\xeb\xd4\xd3\xf3'  JJ\xc0+\xc0/\xc0,\xc00\xcc

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

